# Демонстрация Классификации звонков

Транскрибация через SR

## Подключение библиотек

In [1]:
import os                              # Функции для работы с файлами
import shutil
import pandas as pd                    # Пандас
import re                              # Работа с регулярными выражениями
import numpy as np
import pickle as pkl
import time
import tensorflow as tf
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.saving import load_model
from tensorflow.keras import utils
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt        # Для прорисовки и визуализации

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

from google.colab import data_table
data_table.enable_dataframe_formatter()

!pip install pydub                  #  Для просмотра некоторых аудио параметров
!pip install SpeechRecognition      #  Сервис распознавания речи
!pip install ffmpeg-python          #  Библиотека для обработки звука.

import speech_recognition as sR        # Библиотека для распознавания речи
import ffmpeg                          # Библиотека для обработки аудио.
from pydub import AudioSegment as aS   # Библиотека для предобработки и изменения форматов аудио файлов
from IPython.display import Audio      # Проигрыватель для аудио
import librosa                         # Обработка звука, извлечение параметров

from tqdm.notebook import tqdm
from speech_recognition import recognizers



In [2]:
!pip install git+https://github.com/openai/whisper.git
import whisper

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-eov_hso4
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-eov_hso4
  Resolved https://github.com/openai/whisper.git to commit e58f28804528831904c3b6f2c0e473f346223433
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
from tqdm import trange, tqdm
path_dir = '/content/drive/MyDrive/Media108/'                    # Общий путь для сохранения результатов

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

## Процедуры

In [4]:
def df_to_ohe(df,collist,lst_encoders = {}):
    '''
 Процедура поочередного преобразования колонок из списка через OneHotEncoder в ОНЕ
 С последующей сборкой в единый массив. Параметры:
 df - датафрейм
 collist - список колонок
 Возвращает собранный массив и список энкодеров
    '''
    create_encode = True if len(lst_encoders) == 0 else False
    #print(create_encode)
    list_code = []
    for i in range(len(collist)):
        if create_encode:
            print(f'Формирование OneHotEncoder и кодировка колонки {collist[i]}', end='')
            encoder = OneHotEncoder(handle_unknown='infrequent_if_exist',sparse_output=False)
            encoder.fit(np.array(df[collist[i]].values).reshape(-1, 1))
            lst_encoders[collist[i]] = encoder
            list_code.append(encoder.transform(np.array(df[collist[i]].values).reshape(-1, 1)))
            print(' - Успешно')
        else:
            #print(f'Кодируется колонка {collist[i]}', end='')
            encoder = lst_encoders[collist[i]]
            list_code.append(encoder.transform(np.array(df[collist[i]].values).reshape(-1, 1)))
            #print(' - Успешно')

    x_data = np.hstack(list_code)
    return x_data, lst_encoders

In [5]:
def file_to_df(df,path_audio='Non'):
    '''
 Функция добаввляет к датафрейму поле "audiofile"
 в которое заносит имя айдиофайла связанного с данной записью
 df - датафрейм
 pats_audio - директория с аудиофайлами
 Возвращает измененный датафрейм.
    '''
    if path_audio == 'Non':
        raise KeyError('Не указан каталог с аудио данными.')

    dict_fls = {}
    for files in os.listdir(path_audio):
        session_in_file = re.findall(r'session_(\d*)_talk',files)
        dict_fls[session_in_file[0]] = files

    df.insert(1,'audiofile',value=np.NaN)
    for k in range(len(df)):
        session = str(df['Идентификатор сессии звонка'][k])
        df.loc[k,['audiofile']] = dict_fls[session]

    df.audiofile = df.audiofile.fillna('Аудиофайл_не_найден')

    return df

In [6]:
def recognizeAudio(df, path_audio='',
                   offset=None,
                   duration=None):

    for i in trange(len(df),desc='Транскрибация аудиофайлов:'):
        rezalt = ''
        filname = df['audiofile'][i]
        if filname == 'Аудиофайл_не_найден':
            rezalt = 'безответа'
        else:
            filewav = aS.from_mp3(path_audio+filname).export('file_wav.wav', format='wav')
            AUDIO_FILE = os.path.join(filewav.name)       # Задаем путь к аудиофайлу
            r = sR.Recognizer()                       # Создаем объект класса Recognizer
            with sR.AudioFile(AUDIO_FILE) as source:  # Считываем аудиофайл
                r.adjust_for_ambient_noise(source)
                audio = r.record(source)
#                try:
                rezalt = r.recognize_whisper(audio, model='medium',language='russian')
#                    rezalt = r.recognize_google(audio, language='ru',)
                    #rezalt = r.recognize_sphinx(audio, language='ru',)
                    #def recognize_google(self, audio_data, key=None, language="en-US", pfilter=0, show_all=False, with_confidence=False)
#                except Exception as e:
#                    razalt = 'Транскрибация завершилась ошибкой'

        df.loc[i,['textsr']] = rezalt

    return df

In [7]:
def model_pred(df_path, path_audio='',proces=True):
    '''
 Функция для получения предсказания модели
 по переданным данным:
 df_path - Путь к CSV файлу (файл без заголовочной части, если это не возможно - внесем изменения)
 path_audio - Директория а аудиофайлами (если не передана или передана пустой -
              предполагается что транскрибированные данный переданы в датафрейме в колонке "whisper")
 proces - переменная указавающая нужно ли дополнительно обрабатывать колонки
        (очистка теста, заполнение пустот, перевод длительности разговора в сек.)
 Возвращает список с предсказаниями для всего CSV файла и сам датафрейм.
    '''
    # Загрузка предобученной модели.
    model = load_model('/content/drive/MyDrive/Media108/model9_21.h5')
    # Загрузка датафрейма
    df = pd.read_csv(df_path)
    # Загрузка токенайзера и энкодеров необходимых для подготовки данных
    with open('/content/drive/MyDrive/Media108/token_encoders.pkl', 'rb') as f:
        encoders,tokenizer_txt = pkl.load(f)

    # Вызов функции транскрибации аудиозаписей
    if path_audio != '':
        df = file_to_df(df,path_audio=path_audio)
        df.insert(2,'textsr',value='безответа')    # Добавление колонки для транскрибированного текста
        df = recognizeAudio(df,path_audio=path_audio)   #

    # Очистка текстовой колонки
    df['textsr'] = df['textsr'].fillna('безответа')    # Заполнение пустот
    df['textsr'] = df['textsr'].apply(lambda s: re.sub("[^А-Яа-я0-9 ]", "", s.lower()))
    # Очистка тех строк где после предыдущей операции остались одни пробелы
    df['textsr'] = df['textsr'].apply(lambda s: s.strip())
    # Заменим опустевшие ячейки текстом "безответа"
    df['textsr'] = np.where(df['textsr'].str.len() < 5,'безответа',df['textsr'])
    # Заменим буквы 'ё' на 'е'
    df['textsr'] = df['textsr'].str.replace('ё', 'е', regex=False)

    # Получение матрицы BOW для колонки с расшифровками
    txt_list = df['textsr'].tolist()  # Список расшифровок звонков
    txt_bow = tokenizer_txt.texts_to_matrix(txt_list)

    # Очистка колонок для ОНЕ
    if proces:
        df['ID посетителя'] = df['ID посетителя'].fillna(0)
        df['ID посетителя'] = df['ID посетителя'].astype(int)
        df['Номер абонента'] = df['Номер абонента'].astype(int)
        df['Номер обращения'] = df['Номер обращения'].astype(int)
        df['Тип посетителя'] = df['Тип посетителя'].str.replace('{','', regex=False)
        df['Тип посетителя'] = df['Тип посетителя'].str.replace('}','', regex=False)
        df['Первая рекламная кампания'] = df['Первая рекламная кампания'].str.replace('{','', regex=False)
        df['Первая рекламная кампания'] = df['Первая рекламная кампания'].str.replace('}','', regex=False)

    # Получение матрицы ОНЕ
    col_to_ohe = ['Статус','Тип','Сайт','Тип посетителя','Сценарий','Операции','Тип устройства','Первая рекламная кампания']
    df[col_to_ohe] = df[col_to_ohe].fillna('Нет данных')   # Дозаполним незаполненные данные
    OHE_data, encoders = df_to_ohe(df,col_to_ohe,lst_encoders = encoders)
    OHE_data = OHE_data.astype('float32')

    # Подготовка Числовых колонок
    if proces:
        df['Чистая длительность разговора'] = df['Чистая длительность разговора'].fillna('00-00-00')
        length_talk = df['Чистая длительность разговора'].to_list()
        len_sec = list()
        for lnt in length_talk:
            match_tlk = re.findall(r'(\d\d)', lnt)
            len_sec.append(int(match_tlk[0])*3600+int(match_tlk[1])*60+int(match_tlk[2]))
        df['Чистая длительность разговора'] = len_sec


    # Массив из числовых колонок
    digit_col = ['Чистая длительность разговора','Номер обращения','ID посетителя']
    df[digit_col] = df[digit_col].fillna(0)       # Дозаполнение если есть незаполненные
    df['ID посетителя'] = df['ID посетителя']/100000000       # Понижение разрядноси колонки для более плавной нормализации
    x_data = np.array(df[digit_col].values)
    x_data = x_data.astype('float32')
    # Нормализация массива
    max_val = x_data.max()
    x_data = x_data / max_val

    y_pred = model.predict({'input_x1':txt_bow,'input_x2':OHE_data,'input_x3':x_data}, batch_size=txt_bow.shape[0])

    df.insert(3,'y_pred',value = np.rint(y_pred))

    df['predict'] = np.where(df.y_pred==1.0,'Целевой','Не целевой')

    return df

## Подготовка данных

В продакшене не нужна

In [ ]:
df_1 = pd.read_csv(path_dir+'df_no_teg.csv')
df_2 = df_1[:50].copy()
#df_2.drop(columns=['Имя файла','whisper'],axis=1,inplace=True)
df_2.to_csv(path_dir+'df_to_test.csv',index=False)

## демонстрация работы моддели

In [8]:
s_time = time.time()
df = model_pred(path_dir+'df_to_test.csv',path_audio=path_dir+'AUDIO/',proces=True)
time_all = int(time.time()-s_time)
print(f'Выполнение завершено. Длительность: {int(time_all//60)} мин и {int(time_all%60)} сек.')

Транскрибация аудиофайлов:: 100%|██████████| 50/50 [40:54<00:00, 49.10s/it]


1/1 [==============================] - 2s 2s/step
Выполнение завершено. Длительность: 41 мин и 1 сек.


In [9]:
sum_audio = int(df['Чистая длительность разговора'].sum())
print(f'Общая длина разговоров:{sum_audio // 60} мин. {sum_audio % 60} сек.')

Общая длина разговоров:129 мин. 35 сек.


In [10]:
df.predict.value_counts()

Целевой       29
Не целевой    21
Name: predict, dtype: int64

In [11]:
df[['predict','whisper','textsr']]

,predict,whisper,textsr
0,Не целевой,Здравствуйте! Вы позвонили в компанию ЭМР-Гру...,здравствуйте вы позвонили в компанию пожалуй...
1,Целевой,Здравствуйте! Вы позвонили в компанию Эмэргру...,здравствуйте вы позвонили в компанию пожалуй...
2,Не целевой,Здравствуйте! Вы позвонили в компанию Эмэр Гр...,здравствуйте вы позвонили в компанию пожалуй...
3,Не целевой,"Здравствуйте, вы позвонили в компанию Эмэргру...",здравствуйте вы позвонили в компанию пожалуй...
4,Целевой,"Здравствуйте, вы позвонили компанию Эмергрупп...",здравствуйте вы позвонили компании пожалуйст...
5,Целевой,"Здравствуйте, вы позвонили компанию Эмэргрупп...",здравствуйте вы позвонили компанию пожалуйст...
6,Не целевой,"Здравствуйте, вы позвонили в компанию ЭМР-Гру...",здравствуйте вы позвонили в компанию пожалуй...
7,Целевой,Здравствуйте! Вы позвонили в компанию MR Grou...,здравствуйте вы позвонили в компанию пожалуй...
8,Не целевой,Здравствуйте! Вы позвонили в компанию Эмергру...,здравствуйте вы позвонили в компанию пожалуй...
9,Не целевой,"Алло. Здравствуйте, компания Мергруз, меня зо...",ал здравствуйте компания меня зовут лариса по...
